# Materials associated with the paper: 

Cao, W., Zahirovic, S., Flament, N., Williams, S., Golonka, J., and Müller, R. D., 2017: Improving global paleogeography since the late Paleozoic using paleobiology, Biogeosciences, 14, 5425-5439. https://doi.org/10.5194/bg-14-5425-2017.

### This Jupyter notebook is used to (1) convert fossil data into shapefile format, (2) reconstruct the data back in time using plate tectonic models

The code in this notebook is written in Python 2.7. It utilizes standard scientific Python modules and the open source module 'pygplates' which provides a Python API to the open source GPlates software (https://www.gplates.org/).

## Import the Python modules

In [29]:
import pygplates
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read the input data

In [30]:
# Read the fossil data
pbdb = pd.read_csv('../Data/Fossil/2500Ma-collections.csv')
pbdb

,collection_no,collection.authorizer,license,collection.reference_no,latdec,lngdec,period,epoch,stage,ma_max,ma_min,lithology1,lithology2,environment,tectonic_setting,geology_comments
0,184,A. Miller,CC BY,13,79.130000,-76.000000,Cambrian,Early/Lower Cambrian,NaN,530.000,513.0000,"""limestone""",NaN,peritidal,NaN,"""thin bedded, glauconitic, arenaceous lst. gre..."
1,186,J. Sepkoski,CC BY,13,52.833333,-119.250000,Cambrian,NaN,NaN,541.000,485.4000,"""limestone""",NaN,"reef, buildup or bioherm",NaN,"""oolitic, sandly ls, thick bedded, carbonate b..."
2,187,J. Sepkoski,CC BY,13,52.833333,-119.250000,Cambrian,Middle Cambrian,NaN,513.000,505.0000,"""limestone""",NaN,"reef, buildup or bioherm",NaN,"""ls, partially x-bedded; some intraformational..."
3,189,J. Sepkoski,CC BY,13,42.410556,-73.525000,Cambrian,Early/Lower Cambrian,NaN,516.000,513.0000,"""limestone""","""shale""",offshore shelf,NaN,"""sh and ls; ls beds up to 15 cm thick, fine gr..."
4,193,J. Sepkoski,CC BY,13,65.000000,-111.500000,Cambrian,Early/Lower Cambrian,NaN,530.000,513.0000,"""limestone""","""shale""",slope,NaN,"""calcareous sh and silty ls. 'basin facies' -u..."
5,196,J. Sepkoski,CC BY,13,52.833333,-119.250000,Cambrian,Early/Lower Cambrian,NaN,516.000,513.0000,"""limestone""",NaN,"reef, buildup or bioherm",NaN,"""platy ls, argillaceous, some oolite, carbonat..."
6,201,J. Sepkoski,CC BY,13,39.163889,-119.833056,Cambrian,Early/Lower Cambrian,NaN,516.000,513.0000,"""limestone""",NaN,sand shoal,NaN,"""argillaceous fine grained ls, back-bank, inne..."
7,202,J. Sepkoski,CC BY,13,39.163889,-119.833056,Cambrian,Early/Lower Cambrian,NaN,516.000,513.0000,"""limestone""",NaN,"reef, buildup or bioherm",NaN,"""coarse bioclastic limestone, shoal water carb..."
8,203,J. Sepkoski,CC BY,51517,-19.629167,136.916667,Cambrian,Early/Lower Cambrian,NaN,516.000,513.0000,wackestone,NaN,"reef, buildup or bioherm",NaN,"""oolitic and xtalline ls, BA3"""
9,208,J. Sepkoski,CC BY,13,38.868889,-117.921111,Cambrian,Furongian,NaN,501.000,485.4000,conglomerate,NaN,offshore shelf,NaN,"""gravity-flow ls. congl. outer shelf, BA5"""


## Convert the data into shapefile format

In [31]:
# Extract data set from the database
recon_time = 45
age_max = 49
age_min = 37

#extract subsets
pbdb_subset = pbdb[(pbdb.ma_max<=age_max) & (pbdb.ma_min>=age_min)] 

# Read the tectonic model of Matthews et al (2016)
static_polygons = '../Data/Matthews++_410-0Ma/StaticPolys/PresentDay_StaticPlatePolygons_Matthews++.shp'
# hence, here we can specify a 'fake' rotiation file to avoid reconstructing the fossils
rotation_filename_for_fossils = '/Users/wenchaocao/Desktop/New_Rotation.rot'
rotation_model = pygplates.RotationModel(rotation_filename_for_fossils)

point_features = []

for index,row in pbdb_subset.iterrows():
    point = pygplates.PointOnSphere(float(row.latdec),float(row.lngdec))
    #print row.LONG
    point_feature = pygplates.Feature()
    point_feature.set_geometry(point)
    point_feature.set_shapefile_attribute('Environ',str(row.environment))
    point_feature.set_shapefile_attribute('lithology1',str(row.lithology1))
    point_feature.set_shapefile_attribute('lithology2',str(row.lithology2))
    point_feature.set_shapefile_attribute('coll_no',str(row.collection_no))
    point_feature.set_shapefile_attribute('coll_auth',str(row['collection.authorizer']))
    point_feature.set_shapefile_attribute('coll_refno',str(row['collection.reference_no']))
    point_feature.set_shapefile_attribute('ma_max',row.ma_max)
    point_feature.set_shapefile_attribute('ma_min',row.ma_min)
    point_feature.set_shapefile_attribute('period',str(row.period))
    point_feature.set_shapefile_attribute('epoch',str(row.epoch))
    point_feature.set_shapefile_attribute('stage',str(row.stage))
    point_feature.set_valid_time(age_max,age_min)
    point_features.append(point_feature)

# The partition points function can then be used as before
partitioned_point_features = pygplates.partition_into_plates(static_polygons,
                                                       rotation_model,
                                                       point_features) 

# If partitioned feature has plate id = 0, we throw it away 
partitioned_point_features_not_zero = []
for point_feature in partitioned_point_features:
    if point_feature.get_reconstruction_plate_id() != 0:
        partitioned_point_features_not_zero.append(point_feature)   

feature_collection = pygplates.FeatureCollection(partitioned_point_features_not_zero)
feature_collection.write('Output/ProcessingData_1/PresentDay_Paleobiology_'+str(recon_time)+'Ma.shp')


## Reconstruct the data back in time

In [32]:
# read rotation file
input_rotation_filename_forReconstruction = ['/Users/wenchaocao/Research/41_Paleobiology_Paleotectonics/Rotations/Global_EB_410-0Ma_GK07_Matthews_etal_PMAG_fixed_crossovers.rot']
rotation_model_forReconstruction = pygplates.RotationModel(input_rotation_filename_forReconstruction)

# output data at reconstructed coordinates as a shapefile
recon_time = 45
reconstructed_point_features = []
pygplates.reconstruct(feature_collection,
                      rotation_model_forReconstruction,
                      'Output/ProcessingData_1/Reconstructed_Paleobiology_'+str(recon_time)+'Ma.shp',
                      recon_time)